This script is for validating shifted altimetry data against ground truth LiDAR. It loads a ATL dataframe and underlying LiDAR file and compares the Z values along the ATL track, producing a "z_compare.csv" file.

In [26]:
from cmath import isnan
from ctypes import ArgumentError
from re import I
from statistics import mean
import pandas as pd
import datetime
import os
import argparse
import io
from csv import writer as csvWriter
import math
import numpy as np
from numpy import power as nppower
import datetime
from scipy import stats
import multiprocessing
import traceback

ATL07_SPOT_SIZE = 6

def meanChiropteraForATL(chiropteraDataFrame, atlX, atlY, spotSize):
    """
    Determines the elevation to be used as the corresponding "chiroptera elevation" for the given ATLAS sensor point.
    This works by first querying the dataframe for any points falling within the spot size radius of the given X and Y, which theoretically represent the center of the area illuminated by the ATLAS sensor's laser.
    """

    relevantPoints = chiropteraDataFrame.query("sqrt(@nppower((`x` - @atlX), 2) + @nppower((`y` - @atlY), 2)) <= @spotSize")
    maxZ = relevantPoints["z"].mean()

    return maxZ

def compareToATL07(chiropteraDataFrame, atl03DataFrame):
    """
    Performs the comparison of the given set of chiroptera data to the given set of ATLAS sensor data.
    """

    cMaxX = chiropteraDataFrame["x"].max()
    cMinX = chiropteraDataFrame["x"].min()
    cMaxY = chiropteraDataFrame["y"].max()
    cMinY = chiropteraDataFrame["y"].min()
   
    atl03InBoundsDataFrame = atl03DataFrame[(atl03DataFrame['X'] >= cMinX) & 
                               (atl03DataFrame['X'] <= cMaxX) & 
                               (atl03DataFrame['Y'] >= cMinY) & 
                               (atl03DataFrame['Y'] <= cMaxY)]
    
    #print(f"atl07InBounds has length {len(atl03InBoundsDataFrame)} ")

    chiropteraElevs = []
    atl03Elevs = []

    #this can take a while... calcChiroptera performs a mean over a spot for each corresponding z value in bounds.
    #one approach is to take every Nth Z value...
    for ind, row in atl03InBoundsDataFrame.iterrows():
        x = row["X"]
        y = row["Y"]
        z = row["Z"]
        #print(f"testing {x,y}")
        chiropteraElev = meanChiropteraForATL(chiropteraDataFrame, x, y, ATL07_SPOT_SIZE)
        #print(f"chiropteraElev: {chiropteraElev}")

        if not isnan(chiropteraElev) and not isnan(z):
            chiropteraElevs.append(chiropteraElev)
            atl03Elevs.append(z)

    dfZs = pd.DataFrame(columns=["c","a"])
    dfZs["c"] = chiropteraElevs
    dfZs["a"] = atl03Elevs
    exportShiftedDataPath = "z_compare.csv"
    dfZs.to_csv(exportShiftedDataPath, index=False, header=False)

In [20]:
#load 1B section as Chiroptera LiDAR DF
#load ATL from good_df 

adf = pd.read_csv('shifted_gt3r_full.csv')
adf = adf.rename(columns={'x': 'X'})
adf = adf.rename(columns={'y': 'Y'})
adf = adf.rename(columns={'heights': 'Z'})
adf = adf.drop(columns=['dt', 'conf', 'stype', ])
adf = adf.iloc[:, :3]
adf

,X,Y,Z
0,600003.157424,9.588030e+06,-0.039307
1,600000.877441,9.588023e+06,-0.049513
2,599998.913746,9.588016e+06,-0.009007
3,599996.564733,9.588009e+06,0.018991
4,599993.988526,9.588001e+06,-0.111855
...,...,...,...
20376,558909.611880,9.460473e+06,-0.105480
20377,558908.355119,9.460469e+06,-0.093788
20378,558906.064342,9.460462e+06,-0.043480
20379,558902.724326,9.460451e+06,0.001625


In [12]:
#df should be chirop_names = ["t", "x", "y", "z"]
df.to_pickle('chiroptera_1b.pkl')
df = pd.read_pickle('chiroptera_1b.pkl')

In [27]:
compareToATL07(df, adf)

In [ ]:
#Visualize Z-Compare
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file
file_path = 'z_compare.csv'
data = pd.read_csv(file_path)

x = data.index
#chirop
y1 = data.iloc[:,0]

#atl
y2 = data.iloc[:,1]

# Plot the data
plt.plot(x, y1, label='Chiroptera')
plt.plot(x, y2, label='ATL07')
plt.legend()

plt.title('Chiroptera and ATL heights, Section 1B (all) ')
plt.grid(True)
plt.show()